In [1]:
#!pip install -q git+https://github.com/deepmipt/bert.git@feat/keras1 tensorflow-gpu==1.14.0rc0 deeppavlov

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'1.14.0-rc0'

In [4]:
# tf.compat.v1.enable_eager_execution()

In [5]:
tf.compat.v1.executing_eagerly()

False

# Paraphrase data

In [6]:
BATCH_SIZE = 64  # both for training and evaluation for now
MAX_LEN = 52  # the longest sequence in the paraphrase dataset, workaroud that make shape of all batch tensors equal
NUM_TRAIN_SAMPLES = 6656  # total of 7202 samples minus ~500 (actually 546) samples for validation

In [7]:
tf.keras.utils.get_file(
    fname='paraphraser.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser.zip',
    cache_subdir='dataset',
    extract=True,
    cache_dir='.'
)

'./dataset/paraphraser.zip'

In [8]:
tf.keras.utils.get_file(
    fname='paraphraser_gold.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser_gold.zip',
    cache_subdir='dataset',
    extract=True,
    archive_format='zip',
    cache_dir='.'
)

'./dataset/paraphraser_gold.zip'

In [9]:
tf.keras.utils.get_file(
    fname='rubert_cased_L-12_H-768_A-12_v1_converted.zip',
    origin='http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1_converted.zip',
    cache_subdir='models',
    extract=True,
    cache_dir='.'
)

'./models/rubert_cased_L-12_H-768_A-12_v1_converted.zip'

In [10]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [11]:
def data_gen(train_data=True):
    from deeppavlov.dataset_readers.paraphraser_reader import ParaphraserReader
    from bert_dp.tokenization import FullTokenizer
    ds = ParaphraserReader().read(data_path='dataset', do_lower_case=False)
    tokenizer = FullTokenizer(vocab_file='models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt', do_lower_case=False)
    tds = ds['train'] if train_data else ds['test']
    for pair, label in tds:
        t1 = tokenizer.tokenize(pair[0])
        t2 = tokenizer.tokenize(pair[1])
        _truncate_seq_pair(t1, t2, 512)
        s = ['[CLS]'] + t1 + ['[SEP]'] + t2 + ['[SEP]']
        yield tokenizer.convert_tokens_to_ids(s), label  # tf.one_hot(label, depth=2, dtype=tf.int32)

In [12]:
train_ds_ = tf.data.Dataset.from_generator(data_gen,
                                           output_types=(tf.int32, tf.int32),
                                           output_shapes=(tf.TensorShape([None]), tf.TensorShape([])))

W0604 12:12:27.811957 140394783242048 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:494: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [13]:
train_ds = train_ds_.take(NUM_TRAIN_SAMPLES)

In [14]:
valid_ds = train_ds_.skip(NUM_TRAIN_SAMPLES)

In [15]:
batched_train_ds = train_ds.padded_batch(batch_size=BATCH_SIZE,
                                         padded_shapes=(MAX_LEN,  # tf.constant([-1], dtype=tf.int64),
                                                        ()),  # 2),
                                         drop_remainder=True)  # fixed batch size for now

In [16]:
batched_valid_ds = valid_ds.padded_batch(batch_size=BATCH_SIZE,
                                         padded_shapes=(MAX_LEN,  # tf.constant([-1], dtype=tf.int64),
                                                        ()),  # 2),
                                         drop_remainder=True)  

In [17]:
test_ds = tf.data.Dataset.from_generator(data_gen,
                                         output_types=(tf.int32, tf.int32),
                                         output_shapes=(tf.TensorShape([None]), tf.TensorShape([])),
                                         args=(False,))

In [18]:
batched_test_ds = test_ds.padded_batch(batch_size=BATCH_SIZE,
                                       padded_shapes=(MAX_LEN,  # tf.constant([-1], dtype=tf.int64),
                                                      ()),  # 2),
                                       drop_remainder=True)  # fixed batch size for now

# Model construction

In [19]:
from bert_dp import bert

In [20]:
# some data to pass through model in order to construct it
bert_layer = bert.BERT(name='bert') # naming is important for compatibility with checkpoints
for s in batched_train_ds:
    r = bert_layer(s[0])
    break

W0604 12:12:41.365532 140394783242048 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/embeddings.py:30: The name tf.keras.initializers.TruncatedNormal is deprecated. Please use tf.compat.v1.keras.initializers.TruncatedNormal instead.

W0604 12:12:41.367872 140394783242048 deprecation.py:506] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:94: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0604 12:12:41.381629 140394783242048 deprecation.py:506] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instru

In [21]:
r

<tf.Tensor 'bert/pooler/dense/Tanh:0' shape=(64, 768) dtype=float32>

In [22]:
paraphrase_detector = tf.keras.Sequential([bert_layer, tf.keras.layers.Dense(1, activation='sigmoid')])

# Training

In [23]:
# from bert_dp.weight_decay_optimizers import AdamW

In [33]:
NUM_EPOCHS = 10
VAL_EVERY_N_BATCHES = 52  # ~50, but better suited for paraphraser dataset
TRAIN_STEPS_PER_EPOCH = NUM_TRAIN_SAMPLES // BATCH_SIZE
VALIDATION_STEPS = 8  # calculated by hand for now (~546 samples in batches of 64 with droppping remainder)

In [25]:
learning_rate = 2e-05
# learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=2e-05,
#                                                               # n. of batches per epoch * ~NUM_EPOCHS for convergence 
#                                                               decay_steps=STEPS_PER_EPOCH * NUM_EPOCHS,
#                                                               end_learning_rate=1e-06)

In [26]:
paraphrase_detector.compile(#optimizer=AdamW(weight_decay=0.01,
#                                             learning_rate=learning_rate,
#                                             epsilon=1e-6),  # as in bert_dp
                            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate,
                                                               epsilon=1e-6),
                            loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                            metrics=[tf.keras.metrics.BinaryAccuracy(),
#                                      tf.keras.metrics.Precision(class_id=1),
#                                      tf.keras.metrics.Recall(class_id=1),
                                     tf.keras.metrics.Precision(),
                                     tf.keras.metrics.Recall()
                                    ])

In [27]:
paraphrase_detector.evaluate(batched_test_ds)

W0604 12:13:25.416664 140394783242048 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0604 12:13:27.730638 140390082516736 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



     29/Unknown - 7s 226ms/step - loss: 0.7481 - binary_accuracy: 0.4343 - precision: 0.6163 - recall: 0.1432

[0.7481017688225056, 0.43426725, 0.61627907, 0.14324324]

In [28]:
bert_layer.load_weights('models/rubert_cased_L-12_H-768_A-12_v1_converted/bert_model').assert_consumed()

In [29]:
paraphrase_detector.evaluate(batched_test_ds)

     29/Unknown - 6s 221ms/step - loss: 0.7133 - binary_accuracy: 0.4957 - precision: 0.6189 - recall: 0.4081

[0.7132613535585075, 0.49568966, 0.61885244, 0.40810812]

In [31]:
history = paraphrase_detector.fit(x=batched_train_ds.repeat(),
                                  y=None,
                                  epochs=NUM_EPOCHS * TRAIN_STEPS_PER_EPOCH // VAL_EVERY_N_BATCHES,  # workaround for more frequent evaluation
                                  callbacks=[tf.keras.callbacks.TensorBoard(batch_size=BATCH_SIZE),
                                             tf.keras.callbacks.ReduceLROnPlateau(monitor='val_precision',
                                                                                  factor=0.5,
                                                                                  patience=3,
                                                                                  min_lr=1e-06)],
                                  validation_data=batched_valid_ds,
                                  steps_per_epoch=VAL_EVERY_N_BATCHES,
                                  validation_steps=VALIDATION_STEPS)

Epoch 1/6
50/50 [==============================] - 29s 570ms/step - loss: 0.6593 - binary_accuracy: 0.6566 - precision: 0.6712 - recall: 0.9596 - val_loss: 0.7017 - val_binary_accuracy: 0.6133 - val_precision: 0.6133 - val_recall: 1.0000
Epoch 2/6
50/50 [==============================] - 23s 454ms/step - loss: 0.6762 - binary_accuracy: 0.6075 - precision: 0.6258 - recall: 0.9144 - val_loss: 0.6880 - val_binary_accuracy: 0.6133 - val_precision: 0.6133 - val_recall: 1.0000
Epoch 3/6
50/50 [==============================] - 23s 469ms/step - loss: 0.6484 - binary_accuracy: 0.6694 - precision: 0.6794 - recall: 0.9724 - val_loss: 0.6928 - val_binary_accuracy: 0.6133 - val_precision: 0.6133 - val_recall: 1.0000
Epoch 4/6
50/50 [==============================] - 23s 460ms/step - loss: 0.6759 - binary_accuracy: 0.6119 - precision: 0.6214 - recall: 0.9582 - val_loss: 0.7176 - val_binary_accuracy: 0.3867 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/6
50/50 [=======================

In [34]:
'f1 = ' + str((2 * history.history['precision'][-1] * history.history['recall'][-1]) / 
              (history.history['precision'][-1] + history.history['recall'][-1]))

'f1 = 0.7842615601816838'

In [35]:
eval_metrics = paraphrase_detector.evaluate(batched_test_ds)

     29/Unknown - 7s 236ms/step - loss: 0.6808 - binary_accuracy: 0.5981 - precision: 0.5981 - recall: 1.0000

In [36]:
'f1 = ' + str((2 * eval_metrics[2] * eval_metrics[3]) / 
              (eval_metrics[2] + eval_metrics[3]))

'f1 = 0.7484828244402932'